# Read in files

In [317]:
import glob

In [318]:
# files = glob.glob('cnn/stories/*.story')
# print(len(files))
# files

In [319]:
files = glob.glob('beatles/*.txt')
files

["beatles/i'll-cry-instead.txt",
 'beatles/hey-jude.txt',
 "beatles/nothin'-shakin'.txt",
 'beatles/i-will.txt',
 'beatles/child-of-nature.txt',
 'beatles/helter-skelter.txt',
 'beatles/i-am-the-walrus.txt',
 'beatles/glad-all-over.txt',
 'beatles/johnny-b.-goode.txt',
 'beatles/christmas-time.txt',
 'beatles/i-call-your-name.txt',
 'beatles/no-reply.txt',
 'beatles/hello,-little-girl.txt',
 'beatles/good-day-sunshine.txt',
 'beatles/clarabella.txt',
 'beatles/act-naturally.txt',
 'beatles/dear-wack!.txt',
 'beatles/chains.txt',
 'beatles/blackbird.txt',
 'beatles/keep-your-hands-off-my-baby.txt',
 'beatles/get-back.txt',
 'beatles/happiness-is-a-warm-gun.txt',
 'beatles/if-i-needed-someone-to-love.txt',
 'beatles/crinsk-dee-night.txt',
 'beatles/commonwealth.txt',
 'beatles/norwegian-wood.txt',
 "beatles/ain't-she-sweet.txt",
 'beatles/lucille.txt',
 'beatles/nowhere-man.txt',
 'beatles/give-peace-a-chance.txt',
 'beatles/from-me-to-you.txt',
 'beatles/help!.txt',
 "beatles/i'll-be-on

In [320]:
files = files[:10]

# Define method for generating text from files

In [321]:
def preprocessor(text):
    # replace all occurences of multiple newlines and replace them
    # with a single newline padded with spaces so it is treated as a
    # token
    text = ' \n '.join(t for t in text.split('\n') if t)
    text = 'songbegin %s songend' % text
    return text

In [322]:
def text_generator(files, splitter=None, preprocessor=None):
    # splitter is responsible for breaking up text into smaller sequences
    # i.e. perhaps by splitting on \n
    for f in files:
        text = open(f).read()
        if preprocessor is not None:
            text = preprocessor(text)
        # remove highlights
        text = text.split('@highlight')[0]
        if splitter is None:
            text = [text]
        else:
            text = splitter(text)
        for t in text:
            if not t or len(t) < 10:
                continue
            yield t

In [323]:
next(text_generator(files, preprocessor=preprocessor))

"songbegin I've got every reason on earth to be mad \n 'Cause I just lost the only girl I had \n If I could get my way \n I'd get myself locked up today \n But I can't, so I'll cry instead \n I've got a chip on my shoulder that's bigger that my feet \n I can't talk to people that I meet \n If I could see you now \n I'd try to make you sad somehow \n But I can't, so I'll cry instead \n Don't want to cry when there's people there \n I get shy when they start to stare \n I'm gonna hide myself away \n But I'll come back again someday \n And when you do you'd better hide all the girls \n I'm gonna break their hearts all round the world \n Yes, I'm gonna break them in two \n And show you what your lovin' man can do \n Until then I'll cry instead songend"

# Initialize tokenizer

In [324]:
gen = text_generator(files, preprocessor=preprocessor)

In [356]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(
    num_words=1000,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t',  # no newline
    oov_token='<unk>')

In [357]:
%%time
tokenizer.fit_on_texts(gen)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 4.67 ms


In [358]:
tokenizer.num_words

1000

In [359]:
tokenizer.document_count

9

In [360]:
len(tokenizer.word_index), tokenizer.word_index

(550,
 {'\n\n': 53,
  '\n\nand': 329,
  '\n\nchristmas': 168,
  '\n\ndo': 427,
  '\n\nfor': 312,
  '\n\ngeorge': 374,
  '\n\ngo': 501,
  '\n\nhe': 462,
  '\n\nhelter': 429,
  '\n\nlook': 129,
  '\n\nlove': 321,
  "\n\nshe's": 179,
  '\n\nthis': 113,
  '\n\nthough': 193,
  '\n\ntried': 535,
  '\n\nwe': 404,
  '\n\nwell': 218,
  '\n\nwhen': 434,
  '\n\nwill': 430,
  '\naah': 131,
  '\nand': 393,
  '\ngo': 38,
  '\ni': 116,
  "\ni'm": 424,
  '\nlook': 431,
  '\nmy': 399,
  '\noh': 189,
  '\none': 198,
  '\nplease': 192,
  '\nspent': 183,
  '\nthe': 409,
  '\nthen': 185,
  "\nthere's": 42,
  '\ntill': 203,
  '\ntried': 416,
  '\nwhy': 418,
  '\nyes': 219,
  "'cause": 432,
  '<unk>': 550,
  'a': 7,
  'about': 176,
  'above': 428,
  'act': 182,
  'afraid\nyou': 230,
  'again': 102,
  "again\n\nain't": 109,
  'again\nchristmas': 37,
  'again\no': 165,
  "ain't": 87,
  'air\nsing': 333,
  'all': 19,
  'always': 319,
  'an': 523,
  'and': 11,
  'answer\nyou': 124,
  'any': 351,
  'anytime': 235

In [361]:
lookfor = ['\n', '<unk>', 'songbegin', 'songend']
for L in lookfor:
    if not L in tokenizer.word_index:
        print(repr(L), 'missing')


'\n' missing
'songbegin' missing
'songend' missing


In [362]:
index_to_word = {v: k for k, v in tokenizer.word_index.items()}
index_to_word[0] = '<pad>'

In [363]:
index_to_word

{0: '<pad>',
 1: 'nah',
 2: 'the',
 3: 'to',
 4: 'you',
 5: 'go',
 6: 'i',
 7: 'a',
 8: 'me',
 9: 'is',
 10: 'hey',
 11: 'and',
 12: 'on',
 13: 'my',
 14: 'time',
 15: 'johnny',
 16: 'here',
 17: 'it',
 18: 'jude\nnah',
 19: 'all',
 20: 'make',
 21: 'your',
 22: 'be',
 23: "i'm",
 24: 'will',
 25: 'nothing',
 26: 'shaking\nbut',
 27: 'leaves',
 28: "don't",
 29: 'her',
 30: 'get',
 31: 'that',
 32: 'glad',
 33: 'let',
 34: 'just',
 35: 'was',
 36: "you\ni'm",
 37: 'again\nchristmas',
 38: '\ngo',
 39: 'out',
 40: 'want',
 41: 'like',
 42: "\nthere's",
 43: 'tell',
 44: 'jude',
 45: 'feel',
 46: 'know',
 47: "didn't",
 48: 'of',
 49: 'in',
 50: 'for',
 51: 'better',
 52: 'when',
 53: '\n\n',
 54: 'hurt',
 55: 'baby',
 56: 'she',
 57: 'but',
 58: 'skelter',
 59: 'can',
 60: 'got',
 61: 'so',
 62: 'see',
 63: 'love',
 64: 'jealous',
 65: 'trees',
 66: 'down',
 67: 'b',
 68: 'dang',
 69: "gilly\nit's",
 70: 'silly\nbut',
 71: 'made',
 72: 'well',
 73: 'his',
 74: "i've",
 75: "can't",
 76:

In [365]:
tokenizer.num_words = min(len(tokenizer.word_index)+1, tokenizer.num_words)

In [366]:
gen = text_generator(files)
text = next(gen)

In [367]:
len(text.split())

266

In [368]:
text

"I am he as you are he as you are me and we are all together.\nSee how they run like pigs from a gun, see how they fly.\nI'm crying.\n\nSitting on a cornflake, waiting for the van to come.\nCorporation tee-shirt, stupid bloody Tuesday.\nMan, you been a naughty boy, you let your face grow long.\nI am the egg man, they are the egg men.\nI am the walrus, coo coo cachoo\n\nMister City Policeman sitting\nPretty little policemen in a row.\nSee how they fly like Lucy in the Sky, see how they run.\nI'm crying, I'm crying.\nI'm crying, I'm crying.\n\nYellow matter custard, dripping from a dead dog's eye.\nCrabalocker fishwife, pornographic priestess,\nBoy, you been a naughty girl you let your knickers down.\nI am the eggman, they are the eggmen.\nI am the walrus, coo coo cachoo\n\nSitting in an English garden waiting for the sun.\nIf the sun don't come, you get a tan\nFrom standing in the English rain.\nI am the egg man, they are the egg men.\nI am the walrus, coo coo cachoo ca coo coo cachoo\n

In [369]:
sequence = tokenizer.texts_to_sequences(text.split('\n'))
sequence

[[6, 550, 458, 550, 4, 411, 458, 550, 4, 411, 8, 11, 550, 411, 19, 550],
 [62, 304, 286, 550, 41, 550, 506, 7, 550, 62, 304, 286, 550],
 [23, 550],
 [],
 [471, 12, 7, 550, 250, 50, 2, 550, 3, 101],
 [550, 550, 550, 550, 550, 550],
 [299, 4, 110, 7, 550, 482, 4, 33, 21, 550, 550, 305],
 [6, 550, 2, 550, 299, 286, 411, 2, 550, 550],
 [6, 550, 2, 550, 550, 550, 550],
 [],
 [550, 550, 550, 471],
 [550, 143, 550, 49, 7, 550],
 [62, 304, 286, 550, 41, 550, 49, 2, 550, 62, 304, 286, 550],
 [23, 550, 23, 550],
 [23, 550, 23, 550],
 [],
 [550, 550, 550, 550, 506, 7, 550, 550, 550],
 [550, 550, 550, 550],
 [482, 4, 110, 7, 550, 267, 4, 33, 21, 550, 66],
 [6, 550, 2, 550, 286, 411, 2, 550],
 [6, 550, 2, 550, 550, 550, 550],
 [],
 [471, 49, 523, 550, 550, 250, 50, 2, 120],
 [313, 2, 120, 28, 101, 4, 30, 7, 550],
 [506, 550, 49, 2, 550, 550],
 [6, 550, 2, 550, 299, 286, 411, 2, 550, 550],
 [6, 550, 2, 550, 550, 550, 550, 550, 550, 550, 550],
 [],
 [550, 550, 550, 550, 550],
 [28, 4, 550, 2, 550, 55

In [370]:
[[index_to_word[i] for i in L] for L in sequence]

[['i',
  '<unk>',
  'he',
  '<unk>',
  'you',
  'are',
  'he',
  '<unk>',
  'you',
  'are',
  'me',
  'and',
  '<unk>',
  'are',
  'all',
  '<unk>'],
 ['see',
  'how',
  'they',
  '<unk>',
  'like',
  '<unk>',
  'from',
  'a',
  '<unk>',
  'see',
  'how',
  'they',
  '<unk>'],
 ["i'm", '<unk>'],
 [],
 ['sitting',
  'on',
  'a',
  '<unk>',
  'waiting',
  'for',
  'the',
  '<unk>',
  'to',
  'come'],
 ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'],
 ['man',
  'you',
  'been',
  'a',
  '<unk>',
  'boy',
  'you',
  'let',
  'your',
  '<unk>',
  '<unk>',
  'long'],
 ['i', '<unk>', 'the', '<unk>', 'man', 'they', 'are', 'the', '<unk>', '<unk>'],
 ['i', '<unk>', 'the', '<unk>', '<unk>', '<unk>', '<unk>'],
 [],
 ['<unk>', '<unk>', '<unk>', 'sitting'],
 ['<unk>', 'little', '<unk>', 'in', 'a', '<unk>'],
 ['see',
  'how',
  'they',
  '<unk>',
  'like',
  '<unk>',
  'in',
  'the',
  '<unk>',
  'see',
  'how',
  'they',
  '<unk>'],
 ["i'm", '<unk>', "i'm", '<unk>'],
 ["i'm", '<unk>', "i'm", 

In [371]:
len(sequence), len(sequence[0])

(41, 16)

In [372]:
from keras.preprocessing.sequence import pad_sequences
pad_sequences(sequence)

array([[  6, 550, 458, 550,   4, 411, 458, 550,   4, 411,   8,  11, 550,
        411,  19, 550],
       [  0,   0,   0,  62, 304, 286, 550,  41, 550, 506,   7, 550,  62,
        304, 286, 550],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  23, 550],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 471,  12,   7, 550, 250,  50,   2,
        550,   3, 101],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 550, 550, 550,
        550, 550, 550],
       [  0,   0,   0,   0, 299,   4, 110,   7, 550, 482,   4,  33,  21,
        550, 550, 305],
       [  0,   0,   0,   0,   0,   0,   6, 550,   2, 550, 299, 286, 411,
          2, 550, 550],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   6, 550,   2, 550,
        550, 550, 550],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  0,   0,   0,   0,   

In [373]:
[[index_to_word[i] for i in x] for x in pad_sequences(sequence)]

[['i',
  '<unk>',
  'he',
  '<unk>',
  'you',
  'are',
  'he',
  '<unk>',
  'you',
  'are',
  'me',
  'and',
  '<unk>',
  'are',
  'all',
  '<unk>'],
 ['<pad>',
  '<pad>',
  '<pad>',
  'see',
  'how',
  'they',
  '<unk>',
  'like',
  '<unk>',
  'from',
  'a',
  '<unk>',
  'see',
  'how',
  'they',
  '<unk>'],
 ['<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  "i'm",
  '<unk>'],
 ['<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  'sitting',
  'on',
  'a',
  '<unk>',
  'waiting',
  'for',
  'the',
  '<unk>',
  'to',
  'come'],
 ['<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<un

In [374]:
s = sequence[0]
s

[6, 550, 458, 550, 4, 411, 458, 550, 4, 411, 8, 11, 550, 411, 19, 550]

In [375]:
one_hot = tokenizer.sequences_to_matrix([[i] for i in s])

In [376]:
one_hot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [377]:
# only one per row
import numpy as np
np.argwhere(one_hot == 1)

array([[  0,   6],
       [  1, 550],
       [  2, 458],
       [  3, 550],
       [  4,   4],
       [  5, 411],
       [  6, 458],
       [  7, 550],
       [  8,   4],
       [  9, 411],
       [ 10,   8],
       [ 11,  11],
       [ 12, 550],
       [ 13, 411],
       [ 14,  19],
       [ 15, 550]])

# Define batch generator

In [378]:
# splitter = lambda x: x.split('\n')
splitter = lambda x: [x]

In [379]:
def sequencer(tokens):
    return [tokens[:i] for i in range(1, len(tokens)+1)]

In [380]:
list(sequencer('a quick brown fox'))

['a',
 'a ',
 'a q',
 'a qu',
 'a qui',
 'a quic',
 'a quick',
 'a quick ',
 'a quick b',
 'a quick br',
 'a quick bro',
 'a quick brow',
 'a quick brown',
 'a quick brown ',
 'a quick brown f',
 'a quick brown fo',
 'a quick brown fox']

In [381]:
import random
import numpy as np

def batch_generator(files, tokenizer, maxlen=30, batch_size=32, splitter=None,
                    sequencer=None, preprocessor=None, epoch_end=None):
    batch_sequences = []
    while True:
        random.shuffle(files)
        for text in text_generator(files, splitter, preprocessor):
            tokens = tokenizer.texts_to_sequences([text])[0]
            sequences = sequencer(tokens)
            batch_sequences.extend(sequences)
            
            while len(batch_sequences) >= batch_size:
                X = pad_sequences(batch_sequences[:batch_size], maxlen=maxlen)
                y = tokenizer.sequences_to_matrix([[i] for seq in X for i in seq])
                y = y.reshape((batch_size, maxlen, tokenizer.num_words))
                
                # offset input/output
                X = [X[:-1], X[:-1]]
                y = y[1:]
                
                # reset
                batch_sequences = batch_sequences[batch_size:]
                
                yield X, y
    
        if epoch_end is not None:
            yield epoch_end
                

In [382]:
from keras.preprocessing.text import text_to_word_sequence
batch_gen = batch_generator(files, tokenizer, 30, splitter=splitter, sequencer=sequencer, preprocessor=preprocessor)

In [383]:
X, y = next(batch_gen)

In [384]:
X[0].shape, X[1].shape, y.shape

((31, 30), (31, 30), (31, 30, 551))

In [385]:
x1, x2 = X
x1

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0, 550],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 550,  74],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0, 550,  74,  60],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        550,  74,  60, 146],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 550,
         74,  60, 146, 262],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,

In [386]:
y

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [387]:
[[index_to_word.get(i, '<pad>') for i in x] for x in x1]

[['<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<unk>'],
 ['<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<unk>',
  "i've"],
 ['<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<unk>',
  "i've",
  'got'],
 ['<pad>',


In [388]:
y

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [389]:
y.sum(axis=2)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1

# Training

In [390]:
n_heads = 8
n_layers = 6
d_model = 64*n_heads
vocab_size = tokenizer.num_words
sequence_len = 10
warmup_steps = 200

In [391]:
batch_gen = batch_generator(
    files, tokenizer, sequence_len, splitter=splitter,
    sequencer=sequencer, preprocessor=preprocessor, epoch_end=0)

In [392]:
# loop over batch generator until we hit the end of the epoch
# to calculate number of batches in epoch and compute some
# stats along the way
steps_per_epoch = 0
for batch in batch_gen:
    if batch == 0:
        break
    steps_per_epoch += 1

In [393]:
print('steps per epoch', steps_per_epoch)

steps per epoch 83


In [394]:
train_gen = (X for X in batch_gen if not X == 0)

In [395]:
from keras.callbacks import TerminateOnNaN
callbacks = [TerminateOnNaN()]

In [396]:
from model import Transformer
model = Transformer(
        n_heads=n_heads, encoder_layers=n_layers, decoder_layers=n_layers,
        d_model=d_model, vocab_size=vocab_size, sequence_len=sequence_len,
        layer_normalization=True, dropout=True,
        residual_connections=True)

In [397]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
encoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 512)      282112      encoder_input[0][0]              
                                                                 decoder_input[0][0]              
__________________________________________________________________________________________________
positional_encoding_5 (Position (None, 10, 512)      0           embedding[0][0]                  
          

In [398]:
# import keras.backend as K
# def loss(y_true, y_pred):
#    return K.categorical_crossentropy(y_true[:,-1:,:], y_pred[:,-1:,:])

In [399]:
loss = 'categorical_crossentropy'

In [400]:
class LRScheduler:
    def __init__(self, d_model, warmup_steps):
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.epoch = 1

    def lr(self, epoch):
        lr = self.d_model**-.5 * min(self.epoch**-.5, epoch*(self.warmup_steps**-1.5))
        self.epoch += 1
        return lr
lr_scheduler = LRScheduler(d_model, warmup_steps)

In [401]:
from keras.callbacks import LearningRateScheduler
# callbacks.append(LearningRateScheduler(lr_scheduler.lr))

In [402]:
from keras.optimizers import adam
model.compile(loss=loss, optimizer=adam(lr=1e-4))

In [403]:
# from keras import backend as K
# old_lr = K.get_value(model.optimizer.lr)
# K.set_value(model.optimizer.lr, 1e-4)

In [404]:
n_epochs = 1000
model.fit_generator(
    train_gen, steps_per_epoch=steps_per_epoch,
    epochs=n_epochs, callbacks=callbacks)

Epoch 1/1000
83/83 [==============================] - 31s 373ms/step - loss: 6.1677
Epoch 2/1000
83/83 [==============================] - 13s 155ms/step - loss: 5.8673
Epoch 3/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.6662
Epoch 4/1000
83/83 [==============================] - 13s 155ms/step - loss: 5.5196
Epoch 5/1000
83/83 [==============================] - 13s 157ms/step - loss: 5.4028
Epoch 6/1000
83/83 [==============================] - 13s 158ms/step - loss: 5.3272
Epoch 7/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.2618
Epoch 8/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.1847
Epoch 9/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.1433
Epoch 10/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.0959
Epoch 11/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.0487
Epoch 12/1000
83/83 [==============================] - 13s 154ms/step - lo

83/83 [==============================] - 13s 155ms/step - loss: 3.9942
Epoch 98/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2602
Epoch 99/1000
83/83 [==============================] - 13s 158ms/step - loss: 4.4024
Epoch 100/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.1989
Epoch 101/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.3688
Epoch 102/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.1511
Epoch 103/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.1294
Epoch 104/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.3938
Epoch 105/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.1227
Epoch 106/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2554
Epoch 107/1000
83/83 [==============================] - 13s 158ms/step - loss: 4.3595
Epoch 108/1000
83/83 [==============================] - 13s 154ms/step 

83/83 [==============================] - 13s 157ms/step - loss: 4.2119
Epoch 193/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2916
Epoch 194/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2718
Epoch 195/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2281
Epoch 196/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.0842
Epoch 197/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.4146
Epoch 198/1000
83/83 [==============================] - 13s 158ms/step - loss: 4.1248
Epoch 199/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2627
Epoch 200/1000
83/83 [==============================] - 13s 159ms/step - loss: 4.4229
Epoch 201/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2359
Epoch 202/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2389
Epoch 203/1000
83/83 [==============================] - 13s 157ms/ste

83/83 [==============================] - 13s 156ms/step - loss: 4.4251
Epoch 288/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1203
Epoch 289/1000
83/83 [==============================] - 13s 153ms/step - loss: 4.3348
Epoch 290/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.1197
Epoch 291/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2273
Epoch 292/1000
83/83 [==============================] - 13s 153ms/step - loss: 4.5381
Epoch 293/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.0232
Epoch 294/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.3440
Epoch 295/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2758
Epoch 296/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2068
Epoch 297/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.3359
Epoch 298/1000
83/83 [==============================] - 13s 153ms/ste

83/83 [==============================] - 13s 156ms/step - loss: 4.3498
Epoch 383/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1534
Epoch 384/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.3944
Epoch 385/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2010
Epoch 386/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2862
Epoch 387/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2597
Epoch 388/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2519
Epoch 389/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2110
Epoch 390/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2593
Epoch 391/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.1078
Epoch 392/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.3673
Epoch 393/1000
83/83 [==============================] - 13s 156ms/ste

...

83/83 [==============================] - 13s 158ms/step - loss: 4.3314
Epoch 573/1000
83/83 [==============================] - 13s 152ms/step - loss: 4.3353
Epoch 574/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.0708
Epoch 575/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2803
Epoch 576/1000
83/83 [==============================] - 13s 158ms/step - loss: 4.2928
Epoch 577/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2818
Epoch 578/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.0417
Epoch 579/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.3572
Epoch 580/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.1611
Epoch 581/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2172
Epoch 582/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.3853
Epoch 583/1000
83/83 [==============================] - 13s 155ms/ste

83/83 [==============================] - 13s 155ms/step - loss: 4.2385
Epoch 668/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2458
Epoch 669/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2499
Epoch 670/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2267
Epoch 671/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2065
Epoch 672/1000
83/83 [==============================] - 13s 153ms/step - loss: 4.2969
Epoch 673/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.1871
Epoch 674/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2508
Epoch 675/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2493
Epoch 676/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2372
Epoch 677/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2355
Epoch 678/1000
83/83 [==============================] - 13s 155ms/ste

83/83 [==============================] - 13s 157ms/step - loss: 4.3508
Epoch 763/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2670
Epoch 764/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.3202
Epoch 765/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2152
Epoch 766/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.0144
Epoch 767/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.5115
Epoch 768/1000
83/83 [==============================] - 13s 156ms/step - loss: 3.9304
Epoch 769/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2439
Epoch 770/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.4499
Epoch 771/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1880
Epoch 772/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2040
Epoch 773/1000
83/83 [==============================] - 13s 156ms/ste

83/83 [==============================] - 13s 157ms/step - loss: 4.2370
Epoch 858/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2387
Epoch 859/1000
83/83 [==============================] - 13s 158ms/step - loss: 4.2662
Epoch 860/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1878
Epoch 861/1000
83/83 [==============================] - 13s 152ms/step - loss: 4.2395
Epoch 862/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.1972
Epoch 863/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2763
Epoch 864/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2200
Epoch 865/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2712
Epoch 866/1000
83/83 [==============================] - 13s 153ms/step - loss: 4.2095
Epoch 867/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2809
Epoch 868/1000
83/83 [==============================] - 13s 155ms/ste

83/83 [==============================] - 13s 157ms/step - loss: 3.9397
Epoch 953/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2589
Epoch 954/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.4080
Epoch 955/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1978
Epoch 956/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2281
Epoch 957/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2783
Epoch 958/1000
83/83 [==============================] - 13s 153ms/step - loss: 4.1752
Epoch 959/1000
83/83 [==============================] - 13s 158ms/step - loss: 4.2613
Epoch 960/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.0888
Epoch 961/1000
83/83 [==============================] - 13s 159ms/step - loss: 4.3022
Epoch 962/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1627
Epoch 963/1000
83/83 [==============================] - 13s 154ms/ste

In [78]:
seed = ['songbegin']
tokens = [[tokenizer.word_index[i] for i in seed]]
s = pad_sequences(tokens, maxlen=sequence_len)
for _ in range(30):
    X = [s, s]
    y_hat = model.predict(X)
    i = np.argmax(y_hat[0][-1])
    tokens[0].append(i)
    s = pad_sequences(tokens, maxlen=sequence_len)
' '.join([index_to_word.get(x, '<pad>') for x in tokens[0]])

'songbegin \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n'

In [ ]:
seed = ['you', 'are']
tokens = [[tokenizer.word_index[i] for i in seed]]
print(tokens)
s = pad_sequences(tokens, maxlen=sequence_len)
print(s)

In [ ]:
X, y = next(train_gen)

In [ ]:
x1, _ = X
[index_to_word.get(i, '<pad>') for i in x1[-1]]

In [ ]:
# X = [s, s]
y_hat = model.predict(X)
y_hat.shape

In [ ]:
[index_to_word.get(i, '<pad>') for i in np.argmax(y_hat[-1], axis=1)]

In [ ]:
np.argmax(y_hat[0], axis=1)